### After getting the answers from AMT
we aggregate the labelling and check the krippendorff's alpha. <br>
Them, we use dawid skene algorithmn (standard, not fast) to derive the ground truth

In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

# add path
import os 
import sys 
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv
import krippendorff

import pandas as pd
import numpy as np
import seaborn as sns

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


### load the data
Becasue there are three batches of recipe evaluation, so we need to concatenate the files in to one dataframe

In [10]:
#from sync_google.access_gsheet import save_to_google, amt, load_google_df
#df = pd.concat([load_google_df(amt['labeled'], name) for name in ["[2]","[3]","[4]"]])
#df.to_csv('../data/labeled.csv',index=False)

time: 31.2 ms


In [2]:
df = pd.read_csv('../data/labeled.csv')

time: 151 ms


### check the consistence of answers using krippendorff's alpha

In [3]:
def amt_alpha(df,label_convert):
    df = df[['Input.id','WorkerId','Answer.Your answer.label']]
    df = df.assign(label= df['Answer.Your answer.label'].map(label_convert))
    df = df.dropna()
    return pd.pivot_table(df, values = 'label', index = ['WorkerId'], columns =['Input.id'])
label_convert = dict(zip(['1 - Strongly Disagree','2','3','4',
                      '5 - Strongly Agree','Not Sure'], [1,2,3,4,5,np.nan]))
df_alpha = amt_alpha(df, label_convert)
krippendorff.alpha(df_alpha.values)

0.46679154192779304

time: 64.1 ms


### some preprocessing

In [4]:
def amt_EM(df,label_convert):
    df = df[['Input.id','WorkerId','Answer.Your answer.label']]
    df = df.assign(label= df['Answer.Your answer.label'].map(label_convert))
    df = df.dropna()
    df = df[['WorkerId','Input.id','label']].rename({'WorkerId':'Annotator','Input.id':'Question','Annotation':'label'},axis = 1)
    return df

time: 18.4 ms


In [5]:
label_convert = dict(zip(['1 - Strongly Disagree','2','3','4',
                      '5 - Strongly Agree','Not Sure'], ['A1','A2','A3','A4','A5','A6']))
df_EM = amt_EM(df, label_convert)

time: 21.2 ms


In [6]:
df_EM.groupby('Annotator').count()

Question  label
Annotator                      
A1NBHCFZKJGHKW       997    997
A1SX8IVV82M0LW         3      3
A1VE1CAMPSYRHE       216    216
A1YXQDZYCKILTT       858    858
A1ZD8RU6YB0VEU       154    154
A37WDOIQH6JM6V       168    168
A6FE2ZQNFW12V        601    601
Helena                 3      3

time: 25.2 ms


### send the data to standard dawid skene algorithm, to determine the ground truth
we used the implementation of https://github.com/GoodDeeds/Fast-Dawid-Skene <br>

In [74]:
#df_EM.to_csv('../fast_dawid_skene/data/amt2_dataset/crowd.csv',index=False)

time: 198 ms


In [7]:
after_DS = pd.read_csv('../data/predict.csv')
after_DS = after_DS.iloc[1:]
after_DS['id'] = after_DS['Question'].apply(lambda x: int(''.join(filter(str.isdigit, x))))
after_DS['GI'] = after_DS['Annotation'].apply(lambda x: int(''.join(filter(str.isdigit, x))))
after_DS['AMT'] = after_DS['GI'].apply(lambda x: 0 if x>=4 else 1)

time: 69.9 ms


In [8]:
print('originally, there are %d recipes'%len(after_DS))
after_DS = after_DS[after_DS.GI!=6]
# some of the recipes are labeled as unknown
print('after dropping the unknowns: %d recipes '%len(after_DS))
print("there are %d positive samples where positive means high GI (HD) "%sum(after_DS.AMT))

originally, there are 1000 recipes
after dropping the unknowns: 990 recipes 
there are 506 positive samples where positive means high GI (HD) 
time: 21.4 ms


### Save, this file is equal to ../data/amt_hGI_pos.pickle

In [17]:
# save_pickle(filename = dir_HugeFiles+'dph/amt_GI_20190701.pickle', obj = dict(zip(after_DS.id,after_DS.AMT)), overwrite=False)

time: 29.7 ms


### Additionally, check what kind of food has a higher inter-rater variability

In [9]:
dic = load_pickle('../data/dic_20190819.pickle')
amt_GI = load_pickle('../data/amt_hGI_pos.pickle')

time: 11.8 s


In [10]:
df_variance = df_alpha.std().sort_values(ascending=False).to_frame().reset_index()
df_variance['id'] = df_variance['Input.id'].apply(lambda x: int(x[10:15]))
df_variance = df_variance[df_variance['id'].apply(lambda x: x in amt_GI.keys())] # onlu include the recipe990
df_variance['name'] = df_variance['id'].apply(lambda x: dic[x]['name'])
def fillna(values):
    return values[~np.isnan(values)].tolist()
df_variance['labels'] = df_variance['Input.id'].apply(lambda x: fillna(df_alpha[x].values))

time: 2.03 s


In [11]:
# alpha remains the same after eliminating 10 recipes
df_alpha990 = df_alpha[['RecipeID: %05d<br>' %i for i in amt_GI.keys()]]
krippendorff.alpha(df_alpha990.values)

0.4712687007830718

time: 42.3 ms


In [12]:
# cocktail are harder to classify
df_cocktail = df_variance[df_variance['name'].apply(lambda x: 'cocktail' in x)]
print('cocktail', df_cocktail[0].mean())
print('recipe990', df_variance[0].mean())

cocktail 1.1327274501523443
recipe990 0.7231774765640567
time: 20.9 ms


In [13]:
# whole wheat are harder to classify
df_ww = df_variance[df_variance['name'].apply(lambda x: 'whole wheat' in x)]
print('whole wheat', df_ww[0].mean())
print('recipe990', df_variance[0].mean())

whole wheat 1.3557330204870983
recipe990 0.7231774765640567
time: 18.2 ms


In [14]:
df_variance.head()

Input.id         0     id                       name  \
1  RecipeID: 12045<br>  2.309401  12045     black russian cocktail   
3  RecipeID: 14104<br>  2.081666  14104  grilled sherry pork chops   
4  RecipeID: 06901<br>  2.081666   6901                mexi hominy   
5  RecipeID: 03770<br>  2.081666   3770        low carb cheesecake   
6  RecipeID: 04059<br>  2.081666   4059     death in the afternoon   

            labels  
1  [5.0, 1.0, 1.0]  
3  [5.0, 2.0, 1.0]  
4  [1.0, 4.0, 5.0]  
5  [4.0, 1.0, 5.0]  
6  [5.0, 1.0, 4.0]

time: 33 ms
